In [1]:
import Pkg
Pkg.add("CSV")
Pkg.add("LinearAlgebra")
Pkg.add("Statistics")
Pkg.add("DataFrames")
using CSV
using Statistics
using LinearAlgebra
using DataFrames

  Updating registry at `/srv/julia/pkg/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/Project.toml`
 [no changes]
  Updating `~/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/Project.toml`
 [no changes]
  Updating `~/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/Project.toml`
 [no changes]
  Updating `~/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/Project.toml`
 [no changes]
  Updating `~/Manifest.toml`
 [no changes]


In [2]:
dataset = CSV.read("housingPriceData.csv");


In [3]:
bedrooms=dataset.bedrooms;
bathrooms=dataset.bathrooms;
sqft_living=dataset.sqft_living/1000;
Y=dataset.price;

function scale_features(s)

   μ = mean(s, dims=1);

   σ = std(s, dims=1);

 

   s_norm = (s .- μ) ./ σ;

 

    return (s_norm)

end
bedrooms=scale_features(bedrooms);
bathrooms=scale_features(bathrooms);
sqft_living=scale_features(sqft_living);


In [4]:
trainset = round(0.6*length(bedrooms));
crossvalidationset = round(0.2*length(bedrooms));
bedrooms_training = bedrooms[1:Int(trainset)];
bedrooms_validation = bedrooms[1+Int(trainset):1+Int(trainset)+Int(crossvalidationset)];
bedrooms_test = bedrooms[2+Int(trainset)+Int(crossvalidationset):length(bedrooms)];
bathrooms_training = bathrooms[1:Int(trainset)];
bathrooms_validation = bathrooms[1+Int(trainset):1+Int(trainset)+Int(crossvalidationset)];
bathrooms_test = bathrooms[2+Int(trainset)+Int(crossvalidationset):length(bedrooms)];
sqft_living_training = sqft_living[1:Int(trainset)];
sqft_living_validation = sqft_living[1+Int(trainset):1+Int(trainset)+Int(crossvalidationset)];
sqft_living_test = sqft_living[2+Int(trainset)+Int(crossvalidationset):length(bedrooms)];
Y_training = Y[1:Int(trainset)];
Y_validation = Y[1+Int(trainset):1+Int(trainset)+Int(crossvalidationset)];
Y_test = Y[2+Int(trainset)+Int(crossvalidationset):length(bedrooms)];


In [5]:
m_total = length(bedrooms);
x0_total = ones(m_total);
X_total = cat(x0_total,bedrooms,bathrooms,sqft_living,dims=2);
m_training = length(bedrooms_training);
x0_training = ones(m_training);
X_training = cat(x0_training, bedrooms_training, bathrooms_training, sqft_living_training, dims=2);
m_validation = length(bedrooms_validation);
x0_validation = ones(m_validation);
X_validation = cat(x0_validation, bedrooms_validation, bathrooms_validation, sqft_living_validation, dims=2);
m_test = length(bedrooms_test);
x0_test = ones(m_test);
X_test = cat(x0_test, bedrooms_test, bathrooms_test, sqft_living_test, dims=2);

In [6]:
function costFunctionLasoo(X,Y,Theta,lambda)
    m = length(Y)
    thetan=abs.(Theta)
    thetan[1]=0
    
    cost = (1/(2*m))*sum(((X*Theta)-Y).^2)+(lambda/(2*m))*sum((thetan))
    return cost
end


costFunctionLasoo (generic function with 1 method)

In [8]:
Theta = zeros(4,1);


In [9]:
function gradientDescentLasoo(X,Y,Theta,lr,numIteration,lambda)
    m = length(Y)
    costHistory = zeros(numIteration);
    thetang=(abs.(Theta))./Theta
    thetang[1]=0
    
    for iteration in 1:numIteration
        loss = (X*Theta) - Y;
        gradient = (X' * loss)/m + (lambda/m)*ones(size(Theta)) ;
        Theta = Theta - lr*gradient;
        cost = costFunctionLasoo(X,Y,Theta,lambda);
        costHistory[iteration] = cost;
    end
    return Theta , costHistory 
end

gradientDescentLasoo (generic function with 1 method)

In [12]:
lr =0.01;
lambda = 200;
newTheta ,costHistory = gradientDescentLasoo(X_training,Y_training,Theta,lr,1000,lambda)

([540988.5387039847; -53545.010160098514; 15206.198919284403; 276892.889515892], [2.056828825842131e11, 2.020499943442843e11, 1.9850378819281564e11, 1.9504187469627164e11, 1.9166194123571613e11, 1.8836174920003192e11, 1.8513913129072418e11, 1.819919889336575e11, 1.7891828979327634e11, 1.7591606538504742e11  …  3.3721069407155838e10, 3.3720849739639217e10, 3.372063111146034e10, 3.372041351758723e10, 3.372019695301337e10, 3.3719981412757557e10, 3.3719766891863766e10, 3.3719553385401043e10, 3.371934088846331e10, 3.37191293961693e10])

In [11]:

function cal(X,Y,Theta)
    m = length(Y)
    Ybar=mean(Y)*ones(m)
    RMSE=sqrt(sum((1/m)*((X*Theta-Y).^2)))
    R2=1-(sum((X*Theta-Y).^2)/sum((Y-Ybar).^2))    
    return RMSE,R2
end
RMSE ,R2 = cal(X_test,Y_test,newTheta)

(258519.55627074925, 0.5135709133804711)

In [13]:
Y_Pred_training = X_training*newTheta;
Y_Pred_validation= X_validation*newTheta;
Y_Pred_test = X_test*newTheta;
Y_Pred = X_total*newTheta;

In [14]:
df = DataFrame(Y_Pred)
CSV.write("2b.csv", df)

"2b.csv"